In [1]:
import mysql
import pandas as pd

In [ ]:
import os
os.getcwd()

vars=[f.split("_")[1] for f in os.listdir('scripts') if 'mysql' in f]


def read_vars(file):
    file='mysql_'+file
    fn=os.path.join('scripts',file)
    out=""
    with open(fn,'r') as f:
        out=f.read().strip('\n')
    return out

credentials=dict([(f,read_vars(f)) for f in vars])
credentials.keys()


query=""
pacient_query=""
with open(os.path.join('scripts', 'get_patients.sql'),'r') as f:
    pacient_query=f.read()

with open(os.path.join('scripts', 'get_active.sql'),'r') as f:
    query=f.read()
pacient_query
import mysql.connector


class QueryFactory:
    def connect(self):
        if not(self.conn and self.conn.is_connected()):
            self.conn = mysql.connector.connect(
                host=self.credentials["host"],
                user=self.credentials["user"],
                password=self.credentials["pass"],
                db=self.credentials["db"],
                charset='utf8',
                buffered=True,
                connection_timeout= 180,
                autocommit=True)
            cur = self.conn.cursor()
            cur.execute('set session net_read_timeout=120')
            cur.execute('SET SESSION CHARACTER_SET_RESULTS = latin1')
            cur.close()
            
    def query(self, query):
        self.connect()
        return pd.read_sql(query, con=self.conn)

    def __init__(self,credentials):
        self.conn = None
        self.credentials = credentials



engine = QueryFactory(credentials)

pacient_list =engine.query(pacient_query)
pacient_list

sintomas =engine.query('select idsintomasGeral idsintoma, replace(Nome,";"," ") sintoma, tipo from jsintomasgeral')
sintomas
pacient_ids='{}'.format(list(pacient_list.idpaciente.values)).replace('[','(').replace(']',')')
sintomas_diario_query = "select idsintoma,valor,data,idpaciente from jsintomadiario where data is not null and data != '0000-00-00' and idpaciente in {}".format(pacient_ids)
sintomas_diario_query

In [78]:

sintomas_diario =engine.query(sintomas_diario_query)


,idsintoma,valor,data,idpaciente
0,NaN,NaN,2016-08-19,411
1,14.0,NaN,2016-01-01,456
2,14.0,0.0,2016-08-28,456
3,15.0,NaN,2016-01-01,456
4,15.0,0.0,2016-08-19,226
...,...,...,...,...
601570,397.0,0.0,2020-05-08,1251
601571,402.0,2.0,2020-05-08,1251
601572,408.0,1.0,2020-05-08,1251
601573,410.0,1.0,2020-05-08,1251


In [79]:
sintomas_diario

,idsintoma,valor,data,idpaciente
0,NaN,NaN,2016-08-19,411
1,14.0,NaN,2016-01-01,456
2,14.0,0.0,2016-08-28,456
3,15.0,NaN,2016-01-01,456
4,15.0,0.0,2016-08-19,226
...,...,...,...,...
601570,397.0,0.0,2020-05-08,1251
601571,402.0,2.0,2020-05-08,1251
601572,408.0,1.0,2020-05-08,1251
601573,410.0,1.0,2020-05-08,1251


In [81]:
pacient_data=engine.query('select idpaciente, Nome from jpaciente where idpaciente in {}'.format(pacient_ids))

In [82]:
pacient_data


In [85]:
active_data=sintomas_diario.merge(pacient_data,how='inner',on='idpaciente').merge(sintomas,how='inner',on='idsintoma')


In [87]:
fn=os.path.join('input','active.csv')


In [88]:

active_data.to_csv(fn, sep=';', index=False)
